In [1]:
%%capture
import sys
sys.path.append('/work/pikachu/utils/')
sys.path.append('/work/pikachu/third/')
sys.path.append('..')
from gezi.common import *
import gezi
from src import config
from src.config import *
from src import util
from src.dataset import Dataset
from src.eval import eval_seq, calc_score
from src.tf.decode import adjust_pad
gezi.init_flags()
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
  tf.config.set_visible_devices([physical_devices[x] for x in range(4)], 'GPU')
tf.config.set_soft_device_placement(True)
for gpu in physical_devices:
  tf.config.experimental.set_memory_growth(gpu, True)

In [2]:
%%capture
FLAGS.decode_phrase_type = False
config.init()
record_dir = f'{FLAGS.root}/tfrecords/0.1'
records_pattern = f'{record_dir}/*.tfrec'
files = gezi.list_files(records_pattern) 
FLAGS.valid_files = [x for x in files if int(os.path.basename(x).split('.')[0]) % FLAGS.folds == FLAGS.fold]
dataset = Dataset('valid', files=FLAGS.valid_files)
datas = dataset.make_batch(FLAGS.batch_size, return_numpy=True)
from src.torch.dataset import get_dataloaders
train_loader, eval_loader = get_dataloaders()

In [9]:
models = [
  'final-14layers.finetune',
  'final-14layers.finetune.finetune_epochs-5',
]
model = models[0]
model_dir = f'../working/offline/30/0/{model}'

In [10]:
gezi.restore_configs(model_dir)
FLAGS.pad_rate = 0.9
FLAGS.mn = 'souap'
gezi.try_mkdir('../working/offline/30/0/souap')
model = util.get_model()

In [11]:
lele.load_weights(model, f'{model_dir}/model.pt')

In [12]:
len(eval_loader)

15045

In [13]:
%%capture
num_steps = dataset.num_steps
metrics = eval_seq(eval_loader, model, num_steps)
ic(metrics['score'])

RuntimeError: mat1 and mat2 shapes cannot be multiplied (16384x384 and 769x224)

In [ ]:
ic(metrics['score'])

[08/22/23 21:07:07] 2899217480.py:1 in <module>
                    metrics['score']: 0.7901512124256356


0.7901512124256356

In [ ]:
model_dict = model.state_dict()
soups = {key:[] for key in model_dict}

In [ ]:
for i, model_dir in tqdm(enumerate(models), total=len(models)):
  lele.load_weights(model, f'../working/offline/30/0/{model_dir}/model.pt')
  for k, v in model.state_dict().items():
    soups[k].append(v)
soups = {k:(torch.sum(torch.stack(v), axis = 0) / len(v)).type(v[0].dtype) for k, v in soups.items() if len(v) != 0}
model_dict.update(soups)
model.load_state_dict(model_dict)

  0%|          | 0/3 [00:00<?, ?it/s]

<All keys matched successfully>

In [ ]:
%%capture
metrics = eval_seq(eval_loader, model, num_steps)
ic(metrics['score'])

eval_loop:   0%|          | 0/118 [00:00<?, ?it/s]

[08/22/23 21:19:49] eval.py:186 in eval_seq()
                    len(df): 15045
                    len(df.idx.unique()): 15045
[08/22/23 21:19:49] eval.py:187 in eval_seq()
                    df.head(2):    sequence_id phrase_type  phrase_dup                   phrase_true  \
                                0    342890745     address           1               5301 kifer hill   
                                1    344291439         url           1  https://gowestk.blogspot.com   
                                
                                                    phrase_pred  \
                                0               5301 kifor hill   
                                1  https://gowestk.blogspot.com   
                                
                                                                          phrase_ori  char/max_idx  \
                                0  <PAD><PAD><PAD>5<PAD>33<PAD>0<PAD>1<PAD> <PAD>...            34   
                                1  ht<PAD>

0.7901922926104217